In [1]:
import pandas as pd
from tpot import TPOTRegressor
from sklearn.metrics import r2_score, mean_squared_error

train = pd.read_csv("../../datasets/regression/train.csv")
test = pd.read_csv("../../datasets/regression/test.csv")

### Preprocessing

TPOT library does not handle categorical data. We have to convert the values of some columns by defining a mapping with numeric values.

In [2]:
def drop_null(dataset, drop_cols):
    dataset = dataset.drop(drop_cols, axis = 1)
    dataset = dataset.dropna()
    return dataset

In [3]:
drop_cols = []
for c in train.columns:
    if train[c].isnull().sum() > 40:
        drop_cols.append(c)
train = drop_null(train, drop_cols)
test = drop_null(test, drop_cols)
y_train = train["SalePrice"]
X_train = train.drop(["SalePrice"], axis = 1)

In [4]:
for cat in X_train:
    if X_train[cat].dtype == "object":
        map = dict(zip(X_train[cat].unique(), range(X_train[cat].unique().size)))
        X_train[cat] = X_train[cat].map(map)

for cat in test:
    if test[cat].dtype == "object":
        map = dict(zip(test[cat].unique(), range(test[cat].unique().size)))
        test[cat] = test[cat].map(map)

### Training

In [5]:
tpot = TPOTRegressor(verbosity = 2, max_time_mins = 60)
tpot.fit(X_train.to_numpy().astype("float"), y_train.to_numpy().astype("float"))

c:\Users\micmi\miniconda3\envs\tpot\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -783797431.7596925

Generation 2 - Current best internal CV score: -783797431.7596925

Generation 3 - Current best internal CV score: -781893459.3002961

Generation 4 - Current best internal CV score: -714250179.7772601

Generation 5 - Current best internal CV score: -714250179.7772601

Generation 6 - Current best internal CV score: -714250179.7772601

Generation 7 - Current best internal CV score: -710673551.343764

Generation 8 - Current best internal CV score: -710673551.343764

60.00 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestRegressor(OneHotEncoder(LassoLarsCV(ZeroCount(input_matrix), normalize=True), minimum_fraction=0.25, sparse=False, threshold=10), bootstrap=False, max_features=0.4, min_samples_leaf=3, min_samples_split=7, n_estimators=100)


TPOTRegressor(max_time_mins=60, verbosity=2)

### Evaluation

In [8]:
preds = tpot.predict(test)

In [10]:
y_pred = tpot.predict(X_train)
print(mean_squared_error(y_train, y_pred))

74808044.87432402
